In [1]:
import rawpy
import cv2
from PIL import Image
import numpy as np
from multiprocessing.pool import Pool
import utils
import os


###############################################################################
# Instantiation
###############################################################################
lum_downsample = utils.Downsampling(ratio='4:4:4')
chr_downsample = utils.Downsampling(ratio='4:2:0')
image_block = utils.ImageBlock(block_height=8, block_width=8)
dct2d = utils.DCT2D(norm='ortho')
quantization = utils.Quantization()
zigzagScanning = utils.ZigzagScanning()
entropy = utils.Entropy()
###############################################################################
# Preprocess
###############################################################################
# Read raw image file as array
# Generate random pixel values
raw = rawpy.imread(os.path.join('images', 'DSC05719.ARW'))

    




In [2]:
rgb_img = raw.postprocess()


In [3]:
rgb_img

array([[[123, 119,  61],
        [121, 119,  61],
        [120, 118,  63],
        ...,
        [131, 127,  72],
        [133, 126,  71],
        [133, 128,  69]],

       [[120, 120,  61],
        [120, 119,  61],
        [120, 119,  63],
        ...,
        [131, 123,  74],
        [134, 123,  72],
        [133, 126,  70]],

       [[116, 121,  62],
        [118, 120,  63],
        [121, 118,  63],
        ...,
        [130, 124,  71],
        [133, 124,  70],
        [132, 127,  69]],

       ...,

       [[165, 149,  81],
        [167, 148,  82],
        [169, 150,  82],
        ...,
        [148, 130,  67],
        [151, 131,  66],
        [151, 130,  67]],

       [[166, 149,  80],
        [169, 145,  82],
        [170, 148,  82],
        ...,
        [147, 128,  68],
        [152, 129,  67],
        [152, 129,  68]],

       [[164, 152,  77],
        [168, 149,  78],
        [170, 149,  81],
        ...,
        [147, 129,  68],
        [152, 129,  68],
        [152, 129,  69]]

In [4]:
# Colorspace transform (RGB -> YCrCb)
ycc_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2YCrCb)


In [5]:
ycc_img

array([[[114, 134,  98],
        [113, 134,  99],
        [112, 134, 100],
        ...,
        [122, 134, 100],
        [122, 136,  99],
        [123, 135,  98]],

       [[113, 133,  99],
        [113, 133,  99],
        [113, 133, 100],
        ...,
        [120, 136, 102],
        [120, 138, 101],
        [122, 136,  99]],

       [[113, 130,  99],
        [113, 132, 100],
        [113, 134, 100],
        ...,
        [120, 135, 100],
        [121, 137,  99],
        [122, 135,  98]],

       ...,

       [[146, 142,  91],
        [146, 143,  92],
        [148, 143,  91],
        ...,
        [128, 142,  94],
        [130, 143,  92],
        [129, 144,  93]],

       [[146, 142,  91],
        [145, 145,  92],
        [147, 144,  91],
        ...,
        [127, 142,  95],
        [129, 144,  93],
        [129, 144,  94]],

       [[147, 140,  89],
        [147, 143,  89],
        [148, 144,  90],
        ...,
        [127, 142,  95],
        [129, 144,  94],
        [129, 144,  94]]

In [6]:
# Center
ycc_img = ycc_img.astype(int)-128

In [7]:
ycc_img

array([[[-14,   6, -30],
        [-15,   6, -29],
        [-16,   6, -28],
        ...,
        [ -6,   6, -28],
        [ -6,   8, -29],
        [ -5,   7, -30]],

       [[-15,   5, -29],
        [-15,   5, -29],
        [-15,   5, -28],
        ...,
        [ -8,   8, -26],
        [ -8,  10, -27],
        [ -6,   8, -29]],

       [[-15,   2, -29],
        [-15,   4, -28],
        [-15,   6, -28],
        ...,
        [ -8,   7, -28],
        [ -7,   9, -29],
        [ -6,   7, -30]],

       ...,

       [[ 18,  14, -37],
        [ 18,  15, -36],
        [ 20,  15, -37],
        ...,
        [  0,  14, -34],
        [  2,  15, -36],
        [  1,  16, -35]],

       [[ 18,  14, -37],
        [ 17,  17, -36],
        [ 19,  16, -37],
        ...,
        [ -1,  14, -33],
        [  1,  16, -35],
        [  1,  16, -34]],

       [[ 19,  12, -39],
        [ 19,  15, -39],
        [ 20,  16, -38],
        ...,
        [ -1,  14, -33],
        [  1,  16, -34],
        [  1,  16, -34]]

In [8]:
# Downsampling
Y = lum_downsample(ycc_img[:,:,0])
Cr = chr_downsample(ycc_img[:,:,1])
Cb = chr_downsample(ycc_img[:,:,2])
ycc_img = np.stack((Y, Cr, Cb), axis=2)

In [9]:
Y

array([[-14, -15, -16, ...,  -6,  -6,  -5],
       [-15, -15, -15, ...,  -8,  -8,  -6],
       [-15, -15, -15, ...,  -8,  -7,  -6],
       ...,
       [ 18,  18,  20, ...,   0,   2,   1],
       [ 18,  17,  19, ...,  -1,   1,   1],
       [ 19,  19,  20, ...,  -1,   1,   1]])

In [10]:
Y[0].size

6024

In [11]:
Cr[0].size

6024

In [12]:
Cb.size

24240576

In [13]:
ycc_img

array([[[-14,   6, -29],
        [-15,   6, -29],
        [-16,   6, -28],
        ...,
        [ -6,   8, -26],
        [ -6,   8, -29],
        [ -5,   8, -29]],

       [[-15,   6, -29],
        [-15,   6, -29],
        [-15,   6, -28],
        ...,
        [ -8,   8, -26],
        [ -8,   8, -29],
        [ -6,   8, -29]],

       [[-15,   4, -28],
        [-15,   4, -28],
        [-15,   6, -28],
        ...,
        [ -8,   7, -28],
        [ -7,   8, -29],
        [ -6,   8, -29]],

       ...,

       [[ 18,  14, -36],
        [ 18,  14, -36],
        [ 20,  16, -37],
        ...,
        [  0,  14, -34],
        [  2,  16, -35],
        [  1,  16, -35]],

       [[ 18,  14, -38],
        [ 17,  14, -38],
        [ 19,  17, -38],
        ...,
        [ -1,  12, -33],
        [  1,  16, -34],
        [  1,  16, -34]],

       [[ 19,  14, -38],
        [ 19,  14, -38],
        [ 20,  17, -38],
        ...,
        [ -1,  12, -33],
        [  1,  16, -34],
        [  1,  16, -34]]

In [14]:
# Create 8x8 blocks
blocks, indices = image_block.forward(ycc_img)

In [15]:
blocks[0].size

64

In [16]:
indices[0].size

3

In [17]:
def process_block(block, index):
    
    #Prediction  -> Prediction error
    
    # DCT
    encoded = dct2d.forward(block)
    if index[2] == 0:
        channel_type = 'lum'
    else:
        channel_type = 'chr'
        
    # Quantization
    encoded = quantization.forward(encoded, channel_type)
    
    # RLE + zigzag scanning
    encoded = zigzagScanning.forward(encoded)
    # Entropy coding (Arithmetic)
    encoded, prob = entropy.forward(encoded)
    

    # Reverse Entropy coding (Arithmetic)
    decoded = entropy.backward(encoded, prob)
    # Reverse RLE + zigzag scanning
    decoded = zigzagScanning.backward(encoded)
    # Dequantization
    decoded = quantization.backward(encoded, channel_type)
    
    # Reverse DCT
    compressed = dct2d.backward(decoded)
    return compressed

    #Reverse Prediction 

In [18]:
encoded = dct2d.forward(blocks[0])
if indices[0][2] == 0:
    channel_type = 'lum'
else:
    channel_type = 'chr'
       

In [19]:
encoded

array([[-1.33500000e+02,  1.60049754e+01,  6.13653315e-01,
         4.65166106e-01,  7.50000000e-01,  5.34105645e-01,
         8.28208674e-01,  1.21000555e+00],
       [-1.24328744e+00, -8.84992173e-01, -1.09044081e+00,
         3.17649513e-01,  5.80414582e-02, -7.66873761e-01,
        -4.73542147e-01, -8.22145334e-02],
       [-1.74927192e-01, -3.44874224e-02, -2.73743687e+00,
         1.25839317e-01,  6.92909649e-01, -8.98035554e-01,
         3.03300859e-02, -1.73379981e-01],
       [ 4.88439083e-01, -1.71202903e-01, -5.94797776e-01,
         1.05991936e+00,  2.41688117e-01, -1.21842700e-01,
        -6.09317051e-01, -4.13320371e-01],
       [ 1.25000000e+00,  1.40941690e+00,  1.86417359e+00,
         1.13329629e+00,  5.00000000e-01,  3.06235705e-01,
         5.80824267e-01, -1.01996233e-01],
       [-1.96515645e+00, -8.66796294e-02, -4.78792545e-01,
        -4.75396091e-01,  9.49649629e-01,  7.40807116e-04,
         3.44861335e-01,  3.59038781e-02],
       [ 8.84251366e-01,  3.850590

In [20]:
encoded = quantization.forward(encoded, channel_type)

In [21]:
encoded = zigzagScanning.forward(encoded)

In [22]:
encoded, prob = entropy.forward(encoded)

In [23]:
prob

{-8.0: 0.015625, -0.0: 0.96875, 1.0: 0.015625}

In [24]:
decoded = entropy.backward(encoded, prob)

KeyError: None

In [ ]:
decoded

[]